In [8]:
# Training a small convnet from scratch to classify a single binary label (e.g. agriculture)
# See: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline

pal = sns.color_palette()

import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from datetime import datetime
import time

timestr = time.strftime("%Y%m%d-%H%M%S")
startTime = datetime.now()

Using Theano backend.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
c:\users\me\appdata\local\temp\try_flags_rpdq_c.c:4:19: fatal error: cudnn.h: No such file or directory
compilation terminated.

Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [9]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.recorder import test_import
test_import()

C:\Users\Me\kaggle\amazon
success


In [10]:
data_dir = 'D:/Downloads/amazon/'
target_label = 'agriculture'
num_samples_per_epoch = 60000
# when > 10000 , notebook freezes, use command line
num_epoch = 20
train_dir = data_dir + 'prep/' + target_label + '/train-jpg'
validation_dir = data_dir + 'prep/' + target_label + '/validation-jpg'
batch_size = 64  # limited by GPU memory size and model size
num_validations_per_epoch = 600
# all images will be resized to 128x128, which reduces number of pixels by 4x.
rescaled_dim = 128
model_file = data_dir + 'models/' + target_label + '-' + timestr + '.h5'

In [11]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, rescaled_dim, rescaled_dim)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',  # was binary_crossentropy
              optimizer='adam',
              metrics=['accuracy', 'recall', 'precision'])

In [12]:
# this is the augmentation configuration we will use for training
# TODO add rotation for small data set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.0,
        zoom_range=0.0,
        horizontal_flip=True,
        vertical_flip=True)

# this is the configuration we will use for testing:
# only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolders of train_dir, and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(rescaled_dim, rescaled_dim),  
        batch_size=batch_size,
        class_mode='binary')

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(rescaled_dim, rescaled_dim),
        batch_size=batch_size,
        class_mode='binary')

Found 32383 images belonging to 2 classes.
Found 8096 images belonging to 2 classes.


In [13]:

history = model.fit_generator(
        train_generator,
        samples_per_epoch=num_samples_per_epoch, #  not using all training samples at every epoch.
        nb_epoch=num_epoch,
        validation_data=validation_generator,
        nb_val_samples=num_validations_per_epoch)
model.save(model_file)  # always save your model and weights after training or during training
print('done')

Epoch 1/2
 64/100 [==================>...........] - ETA: 1s - loss: 0.6861 - acc: 0.6094 - recall: 0.1364 - precision: 0.3333

C:\Users\Me\Anaconda2\lib\site-packages\keras\keras\engine\training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


128/100 [======================================] - 15s - loss: 0.6971 - acc: 0.6484 - recall: 0.0682 - precision: 0.1667 - val_loss: 0.6502 - val_acc: 0.6766 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/2
128/100 [======================================] - 12s - loss: 0.6456 - acc: 0.7266 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.6232 - val_acc: 0.6844 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
done


In [14]:
from utils.recorder import record_model_medata

time_spent_trianing = datetime.now() - startTime

record_model_medata(model_file, history, time_spent_trianing)

                                          model_file  accuracy  precision  \
0  D:/Downloads/amazon/models/agriculture-2017050...  0.726562        0.0   

   recall  
0     0.0  


In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for precision
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('model precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for recall
plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.title('model recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [3]:
#from keras.models import load_model
#model = load_model(model_file)
#model = load_model(data_dir + 'models/agriculture-20170505-234342.h5')

In [6]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_10 (Convolution2D) (None, 32, 126, 126)  896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_16 (Activation)       (None, 32, 126, 126)  0           convolution2d_10[0][0]           
____________________________________________________________________________________________________
maxpooling2d_10 (MaxPooling2D)   (None, 32, 63, 63)    0           activation_16[0][0]              
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 32, 61, 61)    9248        maxpooling2d_10[0][0]            
___________________________________________________________________________________________

In [4]:
testset_datagen = ImageDataGenerator(rescale=1./255)

testset_dir = data_dir + 'test'

testset_generator = testset_datagen.flow_from_directory(
        testset_dir,
        target_size=(rescaled_dim, rescaled_dim),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 40669 images belonging to 1 classes.


In [5]:
testset_predict = model.predict_generator(testset_generator, 40669)


In [ ]:
classifier_threshold = 0.5 # TODO https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/32475
y_testset_predictions = (np.array(testset_predict) > classifier_threshold).astype(int)

In [ ]:
print ('total time spent to complete execution:' , datetime.now() - startTime)